In [ ]:
import math
import pandas as pd
import torch
from torch import nn
from d2l import torch as d2l

# 1. 基于位置的前馈网络

### 理解要点：
- 这个FFN是Transformer里每一层（Encoder/Decoder Block）必备模块。
- **Position-wise** 的意思：对序列每个位置，单独做两层MLP，不混合不同token位置的信息。
- 常见结构：Linear → ReLU → Linear
- 作用：提升模型的非线性表达力，相当于给每个位置加“小脑子”。

In [ ]:
class PositionWiseFFN(nn.Module):
    """基于位置的前馈网络（Position-wise Feedforward Network, FFN）"""
    def __init__(self, ffn_num_input, ffn_num_hiddens, ffn_num_outputs, **kwargs):
        super().__init__(**kwargs)
        # 第一层线性变换，输入维->隐藏维
        self.dense1 = nn.Linear(ffn_num_input, ffn_num_hiddens)
        # 激活函数ReLU
        self.relu = nn.ReLU()
        # 第二层线性变换，隐藏维->输出维
        self.dense2 = nn.Linear(ffn_num_hiddens, ffn_num_outputs)

    def forward(self, X):
        # X形状: (batch_size, 序列长度, 输入维)
        # 每个序列位置独立地过两层全连接+激活
        return self.dense2(self.relu(self.dense1(X)))


In [ ]:
ffn = PositionWiseFFN(4, 4, 8)
ffn.eval()
ffn(torch.ones((2, 3, 4)))[0]

# 2. 残差连接和层规范化

In [ ]:
ln = nn.LayerNorm(2)
bn = nn.BatchNorm1d(2)
X = torch.tensor([[1, 2], [2, 3]], dtype=torch.float32)
# LayerNorm按最后一个维度归一化，BatchNorm按batch归一化
print('layer norm:', ln(X), '\nbatch norm:', bn(X))

In [ ]:
class AddNorm(nn.Module):
    """残差连接后做层归一化（Add + LayerNorm）"""
    def __init__(self, normalized_shape, dropout, **kwargs):
        super().__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(normalized_shape)  # 层归一化

    def forward(self, X, Y):
        # Y: 子层输出，X: 残差（原输入）
        # 先做dropout，残差相加，再归一化
        return self.ln(self.dropout(Y) + X)

In [ ]:
add_norm = AddNorm([3, 4], 0.5)
add_norm.eval()
add_norm(torch.ones((2, 3, 4)), torch.ones((2, 3, 4))).shape

# 3. 编码器

- 残差连接：输出=主分支+输入（跳过层），防止深层模型退化。
- LayerNorm：对每个token的全部特征做归一化，提升收敛和稳定性。
- Dropout：正则化，防止过拟合。


In [ ]:
class EncoderBlock(nn.Module):
    """Transformer编码器块（Encoder Block）"""
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
                 dropout, use_bias=False, **kwargs):
        super().__init__(**kwargs)
        # 多头自注意力
        self.attention = d2l.MultiHeadAttention(
            key_size, query_size, value_size, num_hiddens, num_heads, dropout, use_bias)
        # 残差+层归一化
        self.addnorm1 = AddNorm(norm_shape, dropout)
        # 位置前馈网络
        self.ffn = PositionWiseFFN(ffn_num_input, ffn_num_hiddens, num_hiddens)
        self.addnorm2 = AddNorm(norm_shape, dropout)

    def forward(self, X, valid_lens):
        # 1. 多头自注意力 + 残差 + LayerNorm
        Y = self.addnorm1(X, self.attention(X, X, X, valid_lens))
        # 2. FFN + 残差 + LayerNorm
        return self.addnorm2(Y, self.ffn(Y))


In [ ]:
X = torch.ones((2, 100, 24))
valid_lens = torch.tensor([3, 2])
encoder_blk = EncoderBlock(24, 24, 24, 24, [100, 24], 24, 48, 8, 0.5)
encoder_blk.eval()
encoder_blk(X, valid_lens).shape

In [ ]:
class TransformerEncoder(d2l.Encoder):
    """Transformer整体编码器（可多层堆叠）"""
    def __init__(self, vocab_size, key_size, query_size, value_size,
                 num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens,
                 num_heads, num_layers, dropout, use_bias=False, **kwargs):
        super().__init__(**kwargs)
        self.num_hiddens = num_hiddens
        # token embedding层
        self.embedding = nn.Embedding(vocab_size, num_hiddens)
        # 位置编码
        self.pos_encoding = d2l.PositionalEncoding(num_hiddens, dropout)
        # 堆叠多个EncoderBlock
        self.blks = nn.Sequential()
        for i in range(num_layers):
            self.blks.add_module("block"+str(i),
                EncoderBlock(key_size, query_size, value_size, num_hiddens,
                             norm_shape, ffn_num_input, ffn_num_hiddens,
                             num_heads, dropout, use_bias))

    def forward(self, X, valid_lens, *args):
        # 1. embedding并放大，2.加位置编码
        X = self.pos_encoding(self.embedding(X) * math.sqrt(self.num_hiddens))
        # 记录每层的attention weights（可视化分析用）
        self.attention_weights = [None] * len(self.blks)
        # 3. 堆叠block，层层处理
        for i, blk in enumerate(self.blks):
            X = blk(X, valid_lens)
            self.attention_weights[i] = blk.attention.attention.attention_weights
        return X


In [ ]:
encoder = TransformerEncoder(
    200, 24, 24, 24, 24, [100, 24], 24, 48, 8, 2, 0.5)
encoder.eval()
encoder(torch.ones((2, 100), dtype=torch.long), valid_lens).shape

# 4. 解码器

### 结构总结
- 第一层：自注意力（Masked Self-Attention）——保证生成时不会偷看未来。
- 第二层：编码器-解码器注意力（Enc-Dec Attention）——用encoder的信息指导解码。
- 第三层：前馈网络（FFN）——位置无关的非线性映射。
- 每层都用AddNorm残差连接+层归一化，稳定训练。

In [ ]:
class DecoderBlock(nn.Module):
    """Transformer解码器的第i个块"""
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
                 dropout, i, **kwargs):
        super().__init__(**kwargs)
        self.i = i  # 当前是第几层block
        # 第一层：自回归自注意力（mask住未来信息）
        self.attention1 = d2l.MultiHeadAttention(
            key_size, query_size, value_size, num_hiddens, num_heads, dropout)
        self.addnorm1 = AddNorm(norm_shape, dropout)
        # 第二层：编码器-解码器注意力（能看到encoder输出）
        self.attention2 = d2l.MultiHeadAttention(
            key_size, query_size, value_size, num_hiddens, num_heads, dropout)
        self.addnorm2 = AddNorm(norm_shape, dropout)
        # 第三层：位置前馈网络
        self.ffn = PositionWiseFFN(ffn_num_input, ffn_num_hiddens, num_hiddens)
        self.addnorm3 = AddNorm(norm_shape, dropout)

    def forward(self, X, state):
        # state = [enc_outputs, enc_valid_lens, [key_values列表]]
        enc_outputs, enc_valid_lens = state[0], state[1]
        # key_values保存到当前步为止，当前block所有已生成的token表征
        if state[2][self.i] is None:
            key_values = X  # 训练阶段，直接全部输入
        else:
            key_values = torch.cat((state[2][self.i], X), axis=1)  # 推理阶段，拼接历史
        state[2][self.i] = key_values  # 保存回state

        # mask住“未来”的信息（仅能看到当前位置及之前）
        if self.training:
            batch_size, num_steps, _ = X.shape
            # 构造mask矩阵，每一步只能看到前面所有token
            dec_valid_lens = torch.arange(
                1, num_steps + 1, device=X.device).repeat(batch_size, 1)
        else:
            dec_valid_lens = None  # 推理阶段按需mask

        # 1. Decoder的自注意力，输入只看已生成的部分
        X2 = self.attention1(X, key_values, key_values, dec_valid_lens)
        Y = self.addnorm1(X, X2)  # 残差+归一化

        # 2. 编码器-解码器注意力，query是decoder，key/value是encoder输出
        Y2 = self.attention2(Y, enc_outputs, enc_outputs, enc_valid_lens)
        Z = self.addnorm2(Y, Y2)  # 残差+归一化

        # 3. 前馈网络+残差归一化
        return self.addnorm3(Z, self.ffn(Z)), state


In [ ]:
decoder_blk = DecoderBlock(24, 24, 24, 24, [100, 24], 24, 48, 8, 0.5, 0)
decoder_blk.eval()
X = torch.ones((2, 100, 24))
state = [encoder_blk(X, valid_lens), valid_lens, [None]]
decoder_blk(X, state)[0].shape

### 结构总结/重点说明：
- embedding + position encoding：每个token先变向量，再加顺序信息
- 多层DecoderBlock：每层包含自注意力、编码器-解码器注意力、前馈网络，每步都记录attention权重
- 输出层：将隐藏状态转为词表概率分布
- state设计：支持推理时“记住”已生成内容，便于自回归生成

In [ ]:
class TransformerDecoder(d2l.AttentionDecoder):
    """Transformer解码器整体结构"""
    def __init__(self, vocab_size, key_size, query_size, value_size,
                 num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens,
                 num_heads, num_layers, dropout, **kwargs):
        super().__init__(**kwargs)
        self.num_hiddens = num_hiddens
        self.num_layers = num_layers
        # Token embedding
        self.embedding = nn.Embedding(vocab_size, num_hiddens)
        # 位置编码
        self.pos_encoding = d2l.PositionalEncoding(num_hiddens, dropout)
        # 堆叠多个DecoderBlock
        self.blks = nn.Sequential()
        for i in range(num_layers):
            self.blks.add_module("block"+str(i),
                DecoderBlock(key_size, query_size, value_size, num_hiddens,
                             norm_shape, ffn_num_input, ffn_num_hiddens,
                             num_heads, dropout, i))
        # 输出层，把hidden转为词表概率
        self.dense = nn.Linear(num_hiddens, vocab_size)

    def init_state(self, enc_outputs, enc_valid_lens, *args):
        # 返回编码器输出、有效长度、每层block的历史key_values（初始化为None）
        return [enc_outputs, enc_valid_lens, [None] * self.num_layers]

    def forward(self, X, state):
        # X: (batch, seq_len)
        # 1. embedding，放大再加上位置编码
        X = self.pos_encoding(self.embedding(X) * math.sqrt(self.num_hiddens))
        # 初始化attention权重记录，用于分析/可视化
        self._attention_weights = [[None] * len(self.blks) for _ in range (2)]
        # 依次通过每个DecoderBlock
        for i, blk in enumerate(self.blks):
            X, state = blk(X, state)
            # 保存自注意力和encoder-decoder注意力的权重（方便可视化）
            self._attention_weights[0][i] = blk.attention1.attention.attention_weights
            self._attention_weights[1][i] = blk.attention2.attention.attention_weights
        # 输出层，把hidden映射到词表空间
        return self.dense(X), state

    @property
    def attention_weights(self):
        # 返回所有block的注意力权重
        return self._attention_weights


# 5. 训练

In [ ]:
num_hiddens, num_layers, dropout, batch_size, num_steps = 32, 2, 0.1, 64, 10
lr, num_epochs, device = 0.005, 200, d2l.try_gpu()
ffn_num_input, ffn_num_hiddens, num_heads = 32, 64, 4
key_size, query_size, value_size = 32, 32, 32
norm_shape = [32]

train_iter, src_vocab, tgt_vocab = d2l.load_data_nmt(batch_size, num_steps)

encoder = TransformerEncoder(
    len(src_vocab), key_size, query_size, value_size, num_hiddens,
    norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
    num_layers, dropout)
decoder = TransformerDecoder(
    len(tgt_vocab), key_size, query_size, value_size, num_hiddens,
    norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
    num_layers, dropout)
net = d2l.EncoderDecoder(encoder, decoder)
d2l.train_seq2seq(net, train_iter, lr, num_epochs, tgt_vocab, device)

In [ ]:
engs = ['go .', "i lost .", 'he\'s calm .', 'i\'m home .']
fras = ['va !', 'j\'ai perdu .', 'il est calme .', 'je suis chez moi .']
for eng, fra in zip(engs, fras):
    translation, dec_attention_weight_seq = d2l.predict_seq2seq(
        net, eng, src_vocab, tgt_vocab, num_steps, device, True)
    print(f'{eng} => {translation}, ',
          f'bleu {d2l.bleu(translation, fra, k=2):.3f}')

In [ ]:
enc_attention_weights = torch.cat(net.encoder.attention_weights, 0).reshape((num_layers, num_heads, -1, num_steps))
enc_attention_weights.shape

In [ ]:
d2l.show_heatmaps(
    enc_attention_weights.cpu(), xlabel='Key positions',
    ylabel='Query positions', titles=['Head %d' % i for i in range(1, 5)],
    figsize=(7, 3.5))

In [ ]:
dec_attention_weights_2d = [head[0].tolist()
                            for step in dec_attention_weight_seq
                            for attn in step for blk in attn for head in blk]
dec_attention_weights_filled = torch.tensor(pd.DataFrame(dec_attention_weights_2d).fillna(0.0).values)
dec_attention_weights = dec_attention_weights_filled.reshape((-1, 2, num_layers, num_heads, num_steps))
dec_self_attention_weights, dec_inter_attention_weights = dec_attention_weights.permute(1, 2, 3, 0, 4)
dec_self_attention_weights.shape, dec_inter_attention_weights.shape

In [ ]:
# Plusonetoincludethebeginning-of-sequencetoken
d2l.show_heatmaps(
    dec_self_attention_weights[:, :, :, :len(translation.split()) + 1],
    xlabel='Key positions', ylabel='Query positions',
    titles=['Head %d' % i for i in range(1, 5)], figsize=(7, 3.5))

In [ ]:
d2l.show_heatmaps(
    dec_inter_attention_weights, xlabel='Key positions',
    ylabel='Query positions', titles=['Head %d' % i for i in range(1, 5)],
    figsize=(7, 3.5))